In [1]:
import math
import os
os.environ['NUMBA_DISABLE_JIT'] = '1'
import warnings

import numpy as np
import statsmodels.api as sm
from numba import njit
from scipy.optimize import minimize

/home/jose/mambaforge/envs/statsforecast/lib/python3.9/site-packages/statsmodels/compat/pandas.py:65: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import Int64Index as NumericIndex


- [ ] KalmanLike
- [ ] KalmanSmooth
- [x] KalmanFore
- [x] partrans
- [x] ARIMA_undoPars
- [x] ARIMA_transPars
- [x] invpartrans
- [x] ARIMA_Invtrans
- [x] ARIMA_Gradtrans
- [ ] ARIMA_Like
- [x] ARIMA_CSS
- [x] TSconv
- [x] inclu2
- [ ] getQ0bis
- [x] getQ0

In [2]:
@njit
def partrans(p, raw, new):
    if p > 100:
        raise ValueError('can only transform 100 pars in arima0')
        
    new[:p] = np.tanh(raw[:p])
    work = new[:p].copy()
    
    for j in range(1, p):
        a = new[j]
        for k in range(j):
            work[k] -= a * new[j - k - 1]
        new[:j] = work[:j]

In [3]:
@njit
def arima_gradtrans(x, arma):
    eps = 1e-3
    mp, mq, msp = arma[:3]
    n = len(x)
    y = np.identity(n)
    w1 = np.empty(100)
    w2 = np.empty(100)
    w3 = np.empty(100)
    if mp > 0:
        for i in range(mp):
            w1[i] = x[i]
        partrans(mp, w1, w2)
        for i in range(mp):
            w1[i] += eps
            partrans(mp, w1, w3)
            for j in range(mp):
                y[i, j] = (w3[j] - w2[j]) / eps
            w1[i] -= eps
    if msp > 0:
        v = mp + mq
        for i in range(msp):
            w1[i] = x[i + v]
        partrans(msp, w1, w2)
        for j in range(msp):
            w1[i] += eps
            partrans(msp, w1, w3)
            y[i + v, j + v] = (w3[j] - w2[j]) / eps
            w1[i] -= eps
    return y

In [4]:
x = np.array([0.1, 0.4, 1.0, 3.1])
arma = np.array([1, 0, 1])
expected = np.diag([0.9899673, 0.8553135, 1, 1])
np.testing.assert_allclose(arima_gradtrans(x, arma), expected)

In [5]:
@njit
def arima_undopars(x, arma):
    mp, mq, msp = arma[:3]
    n = len(x)
    
    res = x.copy()
    if mp > 0:
        partrans(mp, x, res)
    v = mp + mq
    if msp > 0:
        partrans(msp, x[v:], res[v:])
    return res

In [6]:
expected = np.array([0.09966799, 0.37994896, 1.00000000, 3.10000000])
np.testing.assert_allclose(arima_undopars(x, arma), expected)

In [7]:
@njit
def tsconv(a, b):
    na = len(a)
    nb = len(b)
    
    nab = na + nb - 1
    ab = np.zeros(nab)
    
    for i in range(na):
        for j in range(nb):
            ab[i + j] += a[i] * b[j]
            
    return ab

In [8]:
x = np.arange(1, 11)

In [9]:
tsconv(x, x)

array([  1.,   4.,  10.,  20.,  35.,  56.,  84., 120., 165., 220., 264.,
       296., 315., 320., 310., 284., 241., 180., 100.])

In [10]:
expected_tsconv = np.array([
    1, 4, 10, 20, 35, 56, 84, 120, 165, 220, 264,
    296, 315, 320, 310, 284, 241, 180, 100
])

In [11]:
np.testing.assert_allclose(expected_tsconv, tsconv(x, x))

In [12]:
@njit
def inclu2(np_, xnext, xrow, ynext, d, rbar, thetab):
    for i in range(np_):
        xrow[i] = xnext[i]
    
    ithisr = 0
    for i in range(np_):
        if xrow[i] != 0.:
            xi = xrow[i]
            di = d[i]
            dpi = di + xi * xi
            d[i] = dpi
            cbar = di / dpi
            sbar = xi / dpi
            for k in range(i + 1, np_):
                xk = xrow[k]
                rbthis = rbar[ithisr]
                xrow[k] = xk - xi * rbthis
                rbar[ithisr] = cbar * rbthis + sbar * xk
                ithisr += 1
            xk = ynext
            ynext = xk - xi * thetab[i]
            thetab[i] = cbar * thetab[i] + sbar * xk
            if di == 0.:
                return
        else:
            ithisr = ithisr + np_ - i - 1

In [13]:
@njit
def invpartrans(p, phi, new):
    if p > 100:
        raise ValueError('can only transform 100 pars in arima0')

    new = phi[:p].copy()
    work = new.copy()
    for k in range(p-1):
        j = p - k - 1
        a = new[j]
        for k in range(j):
            work[k] = (new[k] + a * new[j - k - 1]) / (1 - a * a)
        for k in range(j):
            new[k] = work[k]
    for j in range(p):
        new[j] = math.atanh(new[j])

In [14]:
@njit
def arima_undopars(x, arma):
    mp, mq, msp = arma[:3]
    n = len(x)
    
    res = x.copy()
    if mp > 0:
        partrans(mp, x, res)
    v = mp + mq
    if msp > 0:
        partrans(msp, x[v:], res[v:])
    return res

In [15]:
@njit
def ARIMA_invtrans(x, arma):
    mp, mq, msp = arma[:3]
    n = len(x)
    y = x.copy()
    
    if mp > 0:
        invpartrans(mp, x, y)
    v = mp + mq
    if msp > 0:
        invpartrans(msp, x[v:], y[v:])
    return y

In [16]:
x = np.array([0.1, 0.4, 1.0, 3.1])
arma = np.array([1, 0, 1])
ARIMA_invtrans(x, arma)

array([0.1, 0.4, 1. , 3.1])

In [17]:
@njit
def getQ0(phi, theta):
    p = len(phi)
    q = len(theta)
    r = max(p, q + 1)
    
    np_ = r * (r + 1) // 2
    nrbar = np_ * (np_ - 1) // 2
    
    V = np.zeros(np_)
    ind = 0
    for j in range(r):
        vj = 0.
        if j == 0:
            vj = 1.
        elif j - 1 < q:
            vj = theta[j - 1]
        
        for i in range(j, r):
            vi = 0.
            if i == 0:
                vi = 1.0
            elif i - 1 < q:
                vi = theta[i - 1]
            V[ind] = vi * vj
            ind += 1
            
    res = np.zeros((r, r))
    res = res.flatten()
    
    if r == 1:
        if p == 0:
            res[0] = 1.
        else:
            res[0] = 1. / (1. - phi[0] * phi[0])
        
        res = res.reshape((r, r))
        return res
    
    if p > 0:
        rbar = np.zeros(nrbar)
        thetab = np.zeros(np_)
        xnext = np.zeros(np_)
        xrow = np.zeros(np_)
        
        ind = 0
        ind1 = -1
        npr = np_ - r
        npr1 = npr + 1
        indj = npr
        ind2 = npr - 1
        
        for j in range(r):
            phij = phi[j] if j < p else 0.
            xnext[indj] = 0.
            indj += 1
            indi = npr1 + j
            for i in range(j, r):
                ynext = V[ind]
                ind += 1
                phii = phi[i] if i < p else 0.
                if j != r - 1:
                    xnext[indj] = -phii
                    if i != r - 1:
                        xnext[indi] -= phij
                        ind1 += 1
                        xnext[ind1] = -1.
                xnext[npr] = -phii * phij
                ind2 += 1
                if ind2 >= np_:
                    ind2 = 0
                xnext[ind2] += 1.
                inclu2(np_, xnext, xrow, ynext, res, rbar, thetab)
                xnext[ind2] = 0.
                if i != r - 1:
                    xnext[indi] = 0.
                    indi += 1
                    xnext[ind1] = 0.
            
        ithisr = nrbar - 1
        im = np_ - 1
        for i in range(np_):
            bi = thetab[im]
            jm = np_ - 1
            for j in range(i):
                bi -= rbar[ithisr] * res[jm]
                ithisr -= 1
                jm -= 1
            res[im] = bi
            im -= 1
        
        # Now reorder p
        ind = npr
        for i in range(r):
            xnext[i] = res[ind]
            ind += 1
        ind = np_ - 1
        ind1 = npr - 1
        for i in range(npr):
            res[ind] = res[ind1]
            ind -= 1
            ind1 -= 1
        for i in range(r):
            res[i] = xnext[i]
    else:
        indn = np_
        ind = np_
        for i in range(r):
            for j in range(i + 1):
                ind -= 1
                res[ind] = V[ind]
                if j != 0:
                    indn -= 1
                    res[ind] += res[ind]
        
    # Unpack to a full matrix
    ind = np_
    for i in range(r - 1, 0, -1):
        for j in range(r - 1, i - 1, -1):
            ind -= 1
            res[r * i + j] = res[ind]

    for i in range(r - 1):
        for j in range(i + 1, r):
            res[i + r * j] = res[j + r * i]
    
    res = res.reshape((r, r))
    return res

In [18]:
expected_getQ0 = np.array([
       [ -3.07619732,   1.11465544,   2.11357369,   3.15204201,
          4.19013718,   5.22823588,   6.26633453,   7.30443355,
          8.34249459,   9.38458115,  10.        ],
       [  1.11465544,  -3.22931088,   1.92416552,   2.84615733,
          3.80807237,   4.76961073,   5.73115265,   6.69269418,
          7.65427405,   8.61179041,  10.        ],
       [  2.11357369,   1.92416552,  -0.37881633,   5.73654439,
          7.62116681,   9.54570541,  11.46986742,  13.39403227,
         15.31827268,  17.23450038,  20.        ],
       [  3.15204201,   2.84615733,   5.73654439,   4.39470753,
         11.47233269,  14.31920899,  17.20600158,  20.0924165 ,
         22.9789482 ,  25.85347889,  30.        ],
       [  4.19013718,   3.80807237,   7.62116681,  11.47233269,
         11.09276725,  19.13264974,  22.94178352,  26.79083216,
         30.63965504,  34.47249261,  40.        ],
       [  5.22823588,   4.76961073,   9.54570541,  14.31920899,
         19.13264974,  19.71534157,  28.71748151,  33.48887095,
         38.30036514,  43.09150596,  50.        ],
       [  6.26633453,   5.73115265,  11.46986742,  17.20600158,
         22.94178352,  28.71748151,  30.2624308 ,  40.22682604,
         45.96069867,  51.71052289,  60.        ],
       [  7.30443355,   6.69269418,  13.39403227,  20.0924165 ,
         26.79083216,  33.48887095,  40.22682604,  42.73402992,
         53.66094562,  60.32916003,  70.        ],
       [  8.34249459,   7.65427405,  15.31827268,  22.9789482 ,
         30.63965504,  38.30036514,  45.96069867,  53.66094562,
         57.13074521,  68.98805242,  80.        ],
       [  9.38458115,   8.61179041,  17.23450038,  25.85347889,
         34.47249261,  43.09150596,  51.71052289,  60.32916003,
         68.98805242,  73.38026771,  90.        ],
       [ 10.        ,  10.        ,  20.        ,  30.        ,
         40.        ,  50.        ,  60.        ,  70.        ,
         80.        ,  90.        , 100.        ]]
)

In [19]:
x = np.arange(1, 11)
np.testing.assert_allclose(expected_getQ0, getQ0(x, x))

- armaCSS
    - ARIMA_transPars
        - fixed, params narma + ncxreg
        - mask = is.na(fixed)
        - coef = as.double(fixed)
        - arma <- as.integer(c(order[-2L], seasonal$order[-2L], seasonal$period,
                         order[2L], seasonal$order[2L]))
    narma <- sum(arma[1L:4L])
    - ARIMA_CSS


In [20]:
@njit
def arima_transpar(params_in, arma, trans):
    #TODO check trans=True results
    mp, mq, msp, msq, ns = arma[:5]
    p = mp + ns * msp
    q = mq + ns * msq
    
    phi = np.zeros(p)
    theta = np.zeros(q)
    params = params_in.copy()    
    
    if trans:
        n = mp + mq + msp + msq
        if mp > 0:
            partrans(mp, params_in, params)
        v = mp + mq
        if msp > 0:
            partrans(msp, params_in[v:], params[v:])
    if ns > 0:
        phi[:mp] = params[:mp]
        phi[mp:p] = 0.
        theta[:mq] = params[mp:mp+mq]
        theta[mq:q] = 0.
        for j in range(msp):
            phi[(j + 1) * ns - 1] += params[j + mp + mq]
            for i in range(mp):
                phi[(j + 1) * ns + i] -= params[i] * params[j + mp + mq]
        
        for j in range(msq):
            theta[(j + 1) * ns - 1] += params[j + mp + mq + msp]
            for i in range(mq):
                theta[(j + 1) * ns + i] += params[i + mp] * params[j + mp + mq + msp]
    else:
        phi[:mp] = params[:mp]
        theta[:mq] = theta[mp:mp + mq]
        
    return phi, theta

In [21]:
par = np.array([1.26377432,  0.82436223, -0.51341576])
arma = (2, 1, 0, 0, 12, 1, 1)
expected = np.array([0.2748562, 0.6774372]), np.array([-0.5134158])
res = arima_transpar(par, arma, True)
for actual, exp in zip(res, expected):
    np.testing.assert_allclose(actual, exp)

In [22]:
expected_arima_transpar_f = (
    np.array([ 0.5 ,  1.  , -0.25,  0.25, -0.25, -0.25]),
    np.array([0.5 , 1.  , 0.25, 0.75, 0.25, 0.25])
)

In [23]:
params = np.repeat(.5, 10)
arma = np.ones(5, dtype=np.integer) * 2
for exp, calc in zip(expected_arima_transpar_f, arima_transpar(params, arma, False)):
    np.testing.assert_allclose(exp, calc)

In [24]:
@njit
def arima_css(y, arma, phi, theta, ncond):
    n = len(y)
    p = len(phi)
    q = len(theta)
    nu = 0
    ssq = 0.0
    
    w = y.copy()
    
    for i in range(arma[5]):
        for l in range(n - 1, 0, -1):
            w[l] -= w[l - 1]
    
    ns = arma[4]
    for i in range(arma[6]):
        for l in range(n - 1, ns - 1, -1):
            w[l] -= w[l - ns]
    
    resid = np.empty(n)
    resid[:ncond] = 0.
    for l in range(ncond, n):
        tmp = w[l]
        for j in range(p):
            if l - j - 1 < 0:
                continue
            tmp -= phi[j] * w[l - j - 1]
            
        for j in range(min(l - ncond, q)):
            if l - j - 1 < 0:
                continue
            tmp -= theta[j] * resid[l - j - 1]
            
        resid[l] = tmp
        
        if not np.isnan(tmp):
            nu += 1
            ssq += tmp * tmp
    
    res = ssq / nu
    
    return res, resid

In [25]:
arima_css(np.arange(1, 11), 
          np.array([0,0,0,0,0,0,0], dtype=np.int32),
          expected_arima_transpar_f[0],
          expected_arima_transpar_f[1], 
          3)

(0.18831307547433035,
 array([ 0.        ,  0.        ,  0.        ,  0.75      , -0.125     ,
        -0.6875    ,  0.28125   ,  0.015625  , -0.2109375 ,  0.37890625]))

In [26]:
@njit
def _make_arima(phi, theta, delta, kappa = 1e6, tol = np.finfo(float).eps):
    # check nas phi
    # check nas theta
    p = len(phi)
    q = len(theta)
    r = max(p, q + 1)
    d = len(delta)
    
    rd = r + d
    Z = np.concatenate((np.array([1.]), np.zeros(r - 1), delta))
    T = np.zeros((rd, rd))
    
    if p > 0:
        T[:p, 0] = phi
    if r > 1:
        for i in range(1, r):
            T[i - 1, i] = 1

    if d > 0:
        T[r] = Z
        if d > 1:
            # ind = r + np.arange(1, d)
            # T[ind, ind - 1] = 1
            for i in range(d):
                T[r + i - 1, r + i] = 1

    if q < r - 1:
        theta = np.concatenate((theta, np.zeros(r - 1 - q)))

    R = np.concatenate((np.array([1.]), theta, np.arange(d, dtype=np.float64)))
    V = R * R.reshape(-1, 1)
    h = 0.
    a = np.zeros(rd)
    Pn = np.zeros((rd, rd))
    P = np.zeros((rd, rd))
    
    if r > 1:
        Pn[:r, :r] = getQ0(phi, theta)
    else:
        Pn[0, 0] = 1 / (1 - phi[0] ** 2) if p > 0 else 1.
    
    if d > 0:
        for i in range(d):
            Pn[r + i, r + i] = kappa
        
    return phi, theta, delta, Z, a, P, T, V, h, Pn

def make_arima(phi, theta, delta, kappa = 1e6, tol = np.finfo(float).eps):
    keys = ['phi', 'theta', 'delta', 'Z', 'a', 'P', 'T', 'V', 'h', 'Pn']
    res = _make_arima(phi, theta, delta, kappa, tol)
    return dict(zip(keys, res))

In [27]:
y = np.arange(1, 5, dtype=np.float64)
res = make_arima(y, y, y)['T']

In [28]:
def arima_like(y, phi, theta, delta, a, P, Pn, up, use_resid):
    n = len(y)
    rd = len(a)
    p = len(phi)
    q = len(theta)
    d = len(delta)
    r = rd - d
    
    sumlog = 0.
    ssq = 0.
    nu = 0
    
    P = P.ravel()
    Pnew = Pn.ravel()
    anew = np.empty(rd)
    M = np.empty(rd)
    if d > 0:
        mm = np.empty(rd * rd)

    if use_resid:
        rsResid = np.empty(n)
        
    for l in range(n):
        for i in range(r):
            tmp = a[i + 1] if i < r - 1 else 0.
            if i < p:
                tmp += phi[i] * a[0]
            anew[i] = tmp
        if d > 0:
            for i in range(r + 1, rd):
                anew[i] = a[i - 1]
            tmp = a[0]
            for i in range(d):
                tmp += delta[i] * a[r + i]
            anew[r] = tmp
        if l > up:
            if d == 0:
                for i in range(r):
                    vi = 0.
                    if i == 0:
                        vi = 1.
                    elif i - 1 < q:
                        vi = theta[i - 1]
                    for j in range(r):
                        tmp = 0.
                        if j == 0:
                            tmp = vi
                        elif j - 1 < q:
                            tmp = vi * theta[j - 1]
                        if i < p and j < p:
                            tmp += phi[i] * phi[j] * P[0]
                        if i < r - 1 and j < r -1:
                            tmp += P[i + 1 + r * (j + 1)]
                        if i < p and j < r - 1:
                            tmp += phi[i] * P[j + 1]
                        if j < p and i < r -1:
                            tmp += phi[j] * P[i + 1]
                        Pnew[i + r * j] = tmp
            else:
                # mm = TP
                for i in range(r):
                    for j in range(rd):
                        tmp = 0.
                        if i < p:
                            tmp += phi[i] * P[rd * j]
                        if i < r - 1:
                            tmp += P[i + 1 + rd * j]
                        mm[i + rd * j] = tmp
                for j in range(rd):
                    tmp = P[rd * j]
                    for k in range(d):
                        tmp += delta[k] * P[r + k + rd * j]
                    mm[r + rd * j] = tmp
                for i in range(1, d):
                    for j in range(rd):
                        mm[r + i + rd * j] = P[r + i - 1 + rd * j]
                
                # Pnew = mmT'
                for i in range(r):
                    for j in range(rd):
                        tmp = 0.
                        if i < p:
                            tmp += phi[i] * mm[j]
                        if i < r - 1:
                            tmp += mm[rd * (i + 1) + j]
                        Pnew[j + rd * i] = tmp
                for j in range(rd):
                    tmp = mm[j]
                    for k in range(d):
                        tmp += delta[k] * mm[rd * (r + k) + j]
                    Pnew[rd * r + j] = tmp
                for i in range(1, d):
                    for j in range(rd):
                        Pnew[rd * (r + i) + j] = mm[rd * (r + i - 1) + j]
                for i in range(q + 1):
                    vi = 1. if i == 0 else theta[i - 1]
                    for j in range(q + 1):
                        Pnew[i + rd * j] += vi * (1. if j == 0 else theta[j - 1])
    
        if not math.isnan(y[l]):
            #print(anew[0])
            resid = y[l] - anew[0]
            for i in range(d):
                resid -= delta[i] * anew[r + i]
            for i in range(rd):
                tmp = Pnew[i]
                for j in range(d):
                    tmp += Pnew[i + (r + j) * rd] * delta[j]
                M[i] = tmp
            gain = M[0]
            for j in range(d):
                gain += delta[j] * M[r + j]
            if gain < 1e4:
                nu += 1
                ssq += resid * resid / gain
                sumlog += math.log(gain)
            if use_resid:
                 rsResid[l] = resid / math.sqrt(gain)
            for i in range(rd):
                a[i] = anew[i] + M[i] * resid / gain
            for i in range(rd):
                for j in range(rd):
                    P[i + j * rd] = Pnew[i + j * rd] - M[i] * M[j] / gain
        else:
            a[:] = anew[:]
            P[:] = Pnew[:]
            if use_resid:
                rsResid[l] = np.nan
    if use_resid:
        return ssq, sumlog, nu, rsResid
    return ssq, sumlog, nu, 

In [29]:
y = np.arange(10)
phi = np.array([0.99551517])
theta = np.array([])
delta = np.array([1.0])
a = np.array([0., 0.])
P = np.array([[0., 0.], [0., 0.]])
Pn = np.array([
    [5.32878591e+02, 0.00000000e+00],
    [0.00000000e+00, 1.00000000e+06],
])
up = 0
use_resid = True
arima_like(y, phi, theta, delta, a, P, Pn, up, use_resid)

(0.002051882527695168,
 6.270663806368853,
 9,
 array([0.        , 0.04348532, 0.00448483, 0.00448483, 0.00448483,
        0.00448483, 0.00448483, 0.00448483, 0.00448483, 0.00448483]))

In [30]:
@njit
def diff(x, lag, differences):
    y = x.copy()
    for _ in range(differences):
        x = y.copy()
        for i in range(lag):
            y[i] = np.nan
        for i in range(lag, x.size):
            y[i] = x[i] - x[i - lag]
    return y

In [31]:
diff(np.arange(10, dtype=np.float32), 3, 1)

array([nan, nan, nan,  3.,  3.,  3.,  3.,  3.,  3.,  3.], dtype=float32)

In [32]:
def arima(x: np.ndarray,
          order = (0, 0, 0),
          seasonal = {'order': (0, 0, 0), 'period': 1},
          xreg = None,
          include_mean = True,
          transform_pars = True,
          fixed = None,
          init = None,
          method = 'CSS',
          SSinit = 'Gardner1980',
          optim_method = 'BFGS',
          kappa = 1e6):
    SSG = SSinit == 'Gardner1980'
    
    def upARIMA(mod, phi, theta):
        p = len(phi)
        q = len(theta)
        mod['phi'] = phi
        mod['theta'] = theta
        r = max(p, q + 1)
        if p > 0:
            mod['T'][:p, 0] = phi
        if r > 1:
            if SSG:
                mod['Pn'][:r, :r] = getQ0(phi, theta)
            else:
                mod['Pn'][:r, :r] = getQ0bis(phi, theta, tol=0)
        else:
            mod['Pn'][0, 0] = 1 / (1 - phi**2) if p > 0 else 1
        mod['a'][:] = 0  # a es vector?
        return mod
            
    def arimaSS(y, mod):
        # arima_like(y, phi, theta, delta, a, P, Pn, up, use_resid)
        return arima_like(
            y,
            mod['phi'],
            mod['theta'],
            mod['delta'],
            mod['a'],
            mod['P'],
            mod['Pn'],
            0,
            True,
        )
    
    def armafn(p, x, trans):
        par = coef
        par[mask] = p
        trarma = arima_transpar(par, arma, trans)
        Z = upARIMA(mod, trarma[0], trarma[1])
        if Z is None:
            return np.finfo(np.float64).max
        if ncxreg > 0:
            x = x - xreg * par[narma + np.arange(ncxreg)]
        res = arima_like(x,
                         Z['phi'],
                         Z['theta'],
                         Z['delta'],
                         Z['a'],
                         Z['P'],
                         Z['Pn'],
                         0,                        
                         False,
                        )
        s2 = res[0] / res[2]
        return 0.5 * (math.log(s2) + res[1] / res[2])
    
    def arCheck(ar):
        p = np.argmax(np.append(1, -ar) != 0)
        if not p:
            return True
        coefs = np.append(1, -ar[:p])
        roots = np.polynomial.polynomial.polyroots(coefs)
        return all(np.abs(roots) > 1)
    
    def maInvert(ma):
        q = len(ma)
        q0 = np.argmax(np.append(1, ma) != 0)
        if not q0:
            return ma
        coefs = np.append(1, ma[:q0])
        roots = np.polynomial.polynomial.polyroots(coefs)
        ind = np.abs(roots) < 1
        if any(ind):
            return ma
        if q0 == 1:
            return np.append(1 / ma[0], np.repeat(0, q - q0))
        roots[ind] = 1 / roots[ind]
        x = 1
        for r in roots:
            x = np.append(x, 0) - np.append(0, x) / r
        return x.real[1:], np.repeat(0, q - q0)
    
    if x.ndim > 1:
        raise ValueError('Only implemented for univariate time series')
    
    if x.dtype not in (np.float32, np.float64):
        x = x.astype(np.float64)
    n = len(x)
    
    if len(order) != 3 or any(o < 0 or not isinstance(o, int) for o in order):
        raise ValueError('order must be 3 non-negative integers')
    if 'order' not in seasonal:
        raise ValueError('order must be a key in seasonal')
    if len(seasonal['order']) != 3 or any(o < 0 or not isinstance(o, int) for o in seasonal['order']):
        raise ValueError('order must be 3 non-negative integers')
    
    if seasonal['period'] is None or seasonal['period'] == 0:
        warnings.warn('Setting seasonal period to 1')
        seasonal['period'] = 1
    
    #fixed
    #mask 
    arma = (*order[::2], 
            *seasonal['order'][::2],
            seasonal['period'],
            order[1],
            seasonal['order'][1])
    narma = sum(arma[:4])
    
    # xtsp = init x, end x and frequency
    # tsp(x) = None
    Delta = np.array([1.]) 
    for i in range(order[1]):
        Delta = tsconv(Delta, np.array([1., -1.])) 
    
    for i in range(seasonal['order'][1]):
        Delta = tsconv(Delta, np.array([1] + [0]*(seasonal['period'] - 1) + [-1]))
    Delta = - Delta[1:]
    nd = order[1] + seasonal['order'][1]
    n_used = (~np.isnan(x)).sum() - len(Delta)
    
    if xreg is None:
        ncxreg = 0
    else:
        if xreg.shape[0] != n:
            raise Exception('lengths of `x` and `xreg` do not match')
        
        ncxreg = xreg.shape[1]
        
    if include_mean and (nd == 0):
        intercept = np.arange(1, n + 1).reshape(-1, 1)
        if xreg is None:
            xreg = intercept
        else:
            xreg = np.concatenate([intercept, xreg])
            ncxreg += 1
            
    # check nas for method CSS-ML
    if method == 'CSS-ML':
        anyna = np.isnan(x).any()
        if ncxreg:
            anyna |= np.isnan(xreg).any()
        if anyna:
            method = 'ML'
    if method.startswith('CSS'):
        ncond = order[1] + seasonal['order'][1] * seasonal['period']
        ncond1 = order[0] + seasonal['order'][0] * seasonal['period']
        ncond = ncond + ncond1
    else:
        ncond = 0
        
    if fixed is None:
        fixed = np.full(narma + ncxreg, np.nan)
    else:
        if len(fixed) != narma + ncxreg:
            raise Exception('wrong length for `fixed`')
    mask = np.isnan(fixed)

    no_optim = not mask.any()
    
    if no_optim:
        transform_pars = False
        
    if transform_pars:
        ind = arma[0] + arma[1] + np.arange(arma[2])
        # check masks and more
        if any(~mask[np.arange(arma[0])]) or any(~mask[ind]):
            warnings.warn('some AR parameters were fixed: setting transform_pars = False')
            transform_pars = False
        
    init0 = np.zeros(narma)
    parscale = np.ones(narma)
    
    # xreg processing
    if ncxreg:
        cn = xreg.columns
        orig_xreg = (ncxreg == 1) | (~mask[narma + np.arange(ncxreg)]).any()
        if not orig_xreg:
            _, _, vt = np.linalg.svd(xreg[(~np.isnan(xreg)).all(1)])
            xreg = xreg * vt
        dx = x
        dxreg = xreg
        if order[1] > 0:
            dx = diff(dx, 1, order[1])
            dxreg = diff(dxreg, 1, order[1])
        if seasonal['period'] > 1 and seasonal['order'][1] > 0:
            dx = diff(dx, seasonal['period'], seasonal['order'][1])
            dxreg = diff(dxreg, seasonal['period'], seasonal['order'][1])
        if length(dx) > dxreg.shape[1]:
            model = sm.OLS(dx, dxreg)
            result = model.fit()
            fit = {'coefs': result.params, 'stderrs': result.bse}
        else:
            raise RuntimeError
        isna = np.isnan(x) | np.isnan(xreg).any(1)
        n_used = (~isna).sum() - len(Delta)
        init0 = np.append(init0, fit['coefs'])
        ses = fit['stderrs']
        parscale = np.append(parscale, 10 * ses)
        
    if n_used <= 0:
        raise ValueError('Too few non-missing observations')

    if init is not None:
        if len(init) != len(init0):
            raise ValueError(f'init should have length {len(init0)}')
        nan_mask = np.isnan(init)
        if nan_mask.any():
            init[nan_mask] = init0[nan_mask]
        if method == 'ML':
            # check stationarity
            if arma[0] > 0:
                if not arCheck(init[:arma[0]]):
                    raise ValueError('non-stationary AR part')
                if arma[2] > 0:
                    if not arCheck(init[arma[:2]].sum() + np.arange(arma[2])):
                        raise ValueError('non-stationary seasonal AR part')
                if transform_pars:
                    init = ARIMA_invtrans(init, arma)
    else:
        init = init0
            
    def arma_css_op(p):
        phi, theta = arima_transpar(p, arma, False)
        res, resid = arima_css(x, arma, phi, theta, ncond)
        
        return 0.5 * np.log(res)
    
    coef = np.array(fixed)
    # parscale definition, think about it, scipy doesnt use it
    
    if method == 'CSS':
        # add if no_optim result
        res = minimize(arma_css_op, init0, method=optim_method)
        
        if res.status > 0:
            warnings.warn(
                f'possible convergence problem: minimize gave code {res.status}]'
            )
            
        coef[mask] = res.x
        phi, theta = arima_transpar(coef, arma, False)
        mod = make_arima(phi, theta, Delta, kappa)
        if ncxreg > 0:
            x -= np.dot(xreg, coef[narma + np.arange(ncxreg)])
        arimaSS(x, mod)
        val = arima_css(x, arma, phi, theta, ncond)
        sigma2 = val[0]
        var = None if no_optim else res.hess_inv / n_used
    else:
        if method == 'CSS-ML':
            # add if no optim result
            res = minimize(arma_css_op, init[mask], method=optim_method)
            if not res.success:
                warnings.warn(res.message)
            init[mask] = res.x
            if arma[0] > 0:
                if not arCheck(init[:arma[0]]):
                    raise ValueError('non-stationary AR part from CSS')
            if arma[2] > 0:
                if not arCheck(init[arma[:2].sum()] + np.arange(arma[2])):
                    raise ValueError('non-stationary seasonal AR part from CSS')
            ncond = 0
            if transform_pars:
                init = ARIMA_invtrans(init, arma)
                if arma[1] > 0:
                    ind = arma[0] + np.arange(arma[1])
                    init[ind] = maInvert(init[ind])
                if arma[3] > 0:
                    ind = arma[:3].sum() + np.arange(arma[3])
                    init[ind] = maInvert(init[ind])
        trarma = arima_transpar(init, arma, transform_pars)
        mod = make_arima(trarma[0], trarma[1], Delta, kappa, SSinit)
        # add if no optim
        res = minimize(armafn, init[mask], args=(x, transform_pars,), method=optim_method)
        if not res.success:
            warnings.warn(res.message)
        coef[mask] = res.x
        if transform_pars:
            if arma[1] > 0:
                ind = arma[0] + np.arange(arma[1])
                if mask[ind].all():
                    coef[ind] = maInvert(coef[ind])
            if arma[3] > 0:
                ind = arma[:3].sum() + np.arange(arma[3])
                if mask[ind].all():
                    coef[ind] = maInvert(coef[ind])
            if any(coef[mask] != res.x):
                oldcode = res.status
                res = minimize(arma_css_op, coef[mask], method=optim_method)
                res.status = oldcode
                coef[mask] = res.x
            A = arima_gradtrans(coef, arma)
            A = A[mask][mask]
            hess = np.linalg.inv(res.hess_inv)
            sol = np.linalg.solve(hess * n_used, A)
            var = np.dot(sol, sol)
            coef = arima_undopars(coef, arma)
        else:
            var = None if no_optim else np.linalg.inv(res.hess * n_used)
        trarma = arima_transpar(coef, arma, False)
        mod = make_arima(trarma[0], trarma[1], Delta, kappa, SSinit)
        if ncxreg > 0:
            val = arimaSS(np.dot(x - xreg, coef[narma + np.arange(ncxreg)]), mod)
        else:
            val = arimaSS(x, mod)
        sigma2 = val[0] / n_used
        
    value = 2 * n_used * res.fun + n_used + n_used * np.log(2 * np.pi)
    aic = value + 2 * sum(mask) + 2 if method != 'CSS' else np.nan
                              
    nm = []
    if arma[0] > 0: nm.extend([f'ar{i}' for i in range(arma[0])])
    if arma[1] > 0: nm.extend([f'ma{i}' for i in range(arma[1])])
    if arma[2] > 0: nm.extend([f'sar{i}' for i in range(arma[2])])
    if arma[3] > 0: nm.extend([f'sma{i}' for i in range(arma[3])])
    if ncxreg > 0:
        nm.append(cn)
        if not orig_xreg:
            ind = narma + np.arange(ncxreg)
            coef[ind] = np.dot(vt, coef[ind])
            A = np.identity(narma + ncxreg)
            A[ind, ind] = vt
            A = A[mask, mask]
            var = np.dot(np.dot(A, var), A.T)
    coef = dict(zip(nm, coef))
    if no_optim:
        var = pd.DataFrame(var, columns=nm[mask], index=nm[mask])
    resid = val[1]
        
    ans = {
        'coef': coef, 
        'sigma2': sigma2, 
        'var_coef': var, 
        'mask': mask,
        'loglik': -0.5 * value, 
        'aic': aic, 
        'arma': arma,
        'residuals': resid, 
        #'series': series,
        'code': res.status, 
        'n_cond': ncond, 
        'nobs': n_used,
        'model': mod
    }
    return ans

In [33]:
x = np.loadtxt('../test_data/ap.csv')
order = (2, 1, 1)
seasonal = {'order': (0, 1, 0), 'period': 12}
res = arima(x, order, seasonal, method='CSS')
res['coef']

/tmp/ipykernel_57433/3710387620.py:249: UserWarning: possible convergence problem: minimize gave code 2]
  warnings.warn(


{'ar0': 0.6806505466573852,
 'ar1': 0.24123846547567415,
 'ma0': -1.0965395248048528}

In [34]:
res['model']['T'][:5, :5]  # diferente

array([[ 0.68065055,  1.        ,  0.        ,  0.        ,  0.        ],
       [ 0.24123847,  0.        ,  1.        ,  0.        ,  0.        ],
       [ 1.        ,  0.        ,  1.        ,  1.        , -0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  1.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ]])

In [35]:
res['model']['V'][1:5, 1:5]  # diferente

array([[ 1.20239893, -0.        , -1.09653952, -2.19307905],
       [-0.        ,  0.        ,  0.        ,  0.        ],
       [-1.09653952,  0.        ,  1.        ,  2.        ],
       [-2.19307905,  0.        ,  2.        ,  4.        ]])

In [36]:
res['model']['Pn'][1:5, 1:5]  # diferente

array([[ 1.20239893e+00, -2.13409700e-17,  8.05722952e-16,
         8.82092165e-16],
       [-2.12231512e-17, -8.79758173e-17,  3.94430453e-31,
        -1.57772181e-30],
       [ 8.05841112e-16,  3.52778241e-15,  1.87348655e-16,
        -2.18562342e-29],
       [ 8.82213397e-16,  1.63588402e-14,  1.27018223e-14,
        -6.20407862e-19]])

In [37]:
@njit
def kalman_forecast(n, Z, a, P, T, V, h):
    p = len(a)
    
    anew = np.empty(p)
    Pnew = np.empty(p * p)
    mm = np.empty(p * p)
    forecasts = np.empty(n)
    se = np.empty(n)
    
    a = a.flatten()
    P = P.flatten()
    T = T.flatten()
    V = V.flatten()
    
    for l in range(n):
        fc = 0.
        for i in range(p):
            tmp = 0.
            for k in range(p):
                tmp += T[i + p * k] * a[k]
            anew[i] = tmp
            fc += tmp * Z[i]
            
        for i in range(p):
            a[i] = anew[i]
        forecasts[l] = fc
    
        for i in range(p):
            for j in range(p):
                tmp = 0.
                for k in range(p):
                    tmp += T[i + p * k] * P[k + p * j]
                mm[i + p * j] = tmp

        for i in range(p):
            for j in range(p):
                tmp = V[i + p * j]
                for k in range(p):
                    tmp += mm[i + p * k] * T[j + p * k]
                Pnew[i + p * j] = tmp

        tmp = h
        for i in range(p):
            for j in range(p):
                P[i + j * p] = Pnew[i + j * p]
                tmp += Z[i] * Z[j] * P[i + j * p]
                
        se[l] = tmp

    return forecasts, se

In [38]:
kalman_forecast(10, 
                *(res['model'][var] for var in ['Z', 'a', 'P', 'T', 'V', 'h']))

(array([ 1571.70470518,  1409.04200962,  2124.16390334,  4213.08246628,
         7188.90696116, 11884.21777076, 19965.13400499, 33644.1305798 ,
        56707.70996204, 95715.64293115]),
 array([3.11830551e-01, 6.33350917e+00, 2.11252212e+01, 2.67662959e+01,
        5.01046988e+01, 1.18155253e+02, 2.92540494e+02, 7.77046873e+02,
        2.16082181e+03, 6.08432623e+03]))